Schelling's Model

In [1]:
import numpy as np

In [ ]:
class Life():
    def __init__(self,
                 agents_init_fraction,
                 low_threshold=4,
                 high_threshold=7,
                field_x_size=100,
                field_y_size=100):
        '''
        Creates a class fore game of life.
        
        @param agents_init_fraction (list): list for initial amount of units for each agent type.
        @param low_threshold (int): agent wants to move if it has less tham low_threshold neighbors.
        @param high_threshold (int): agent wants to move if it has more tham low_threshold neighbors.
        @param field_x_size (int): width of the map.
        @param field_y_size (int): hight of the map.
    
        '''
        
        assert sum(agents_init_fraction) <= 1        
        self.map = np.zeros((field_y_size, field_x_size))
        
        
        